# Eda Noticias

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import numpy as np
from datetime import datetime

class NewsVisualizer:
    def __init__(self, full_file, theme_analysis_file):
        """
        Inicializa el visualizador con los archivos de datos
        """
        self.full_df = pd.read_csv(full_file)
        self.theme_analysis = pd.read_csv(theme_analysis_file)


        plt.style.use('ggplot')

        self.output_dir = Path('visualizations')
        self.output_dir.mkdir(exist_ok=True)

    def _prepare_date_data(self):
        """Prepara los datos para la visualización temporal"""
        # Convertir la columna de publish_date a datetime
        self.full_df['year'] = pd.to_datetime(self.full_df['publish_date']).dt.year

        # Agrupar por tema y año
        temporal_data = self.full_df.groupby(['theme', 'year']).size().reset_index(name='count')
        return temporal_data

    def create_theme_distribution_bar(self):
        """
        Crea un gráfico de barras horizontal con la distribución de artículos por tema
        """
        # Ordenar temas por cantidad de artículos
        theme_analysis_sorted = self.theme_analysis.sort_values('article_count', ascending=True)

        # Crear figura con tamaño específico
        plt.figure(figsize=(12, 10))

        # Crear gráfico de barras horizontal
        bars = plt.barh(theme_analysis_sorted['theme'],
                       theme_analysis_sorted['article_count'],
                       color=sns.color_palette("husl", len(theme_analysis_sorted)))

        # Agregar etiquetas en las barras
        for bar in bars:
            width = bar.get_width()
            plt.text(width, bar.get_y() + bar.get_height()/2,
                    f'{int(width)}',
                    va='center', ha='left', fontsize=10)

        # Ajustar layout y etiquetas
        plt.title('Distribución de Artículos por Tema', pad=20, fontsize=14)
        plt.xlabel('Número de Artículos', fontsize=12)
        plt.ylabel('Tema', fontsize=12)

        plt.tight_layout()

        # Guardar figura
        plt.savefig(self.output_dir / 'theme_distribution.png',
                   dpi=300,
                   bbox_inches='tight')
        plt.close()

    def create_theme_percentage_pie(self):
        """
        Crea un gráfico circular con los porcentajes por tema
        """
        # Ordenar por porcentaje
        theme_analysis_sorted = self.theme_analysis.sort_values('percentage_of_total', ascending=False)

        # Crear figura
        plt.figure(figsize=(12, 8))

        # Crear gráfico circular
        plt.pie(theme_analysis_sorted['percentage_of_total'],
                labels=theme_analysis_sorted['theme'],
                autopct='%1.1f%%',
                colors=sns.color_palette("husl", len(theme_analysis_sorted)),
                wedgeprops={'edgecolor': 'white'},
                textprops={'fontsize': 8})

        plt.title('Distribución Porcentual de Artículos por Tema', pad=20, fontsize=14)

        plt.tight_layout()

        plt.savefig(self.output_dir / 'theme_percentage.png',
                   dpi=300,
                   bbox_inches='tight')
        plt.close()

    def create_temporal_bubble_chart(self):
        """
        Crea un bubble chart interactivo usando Plotly con mejores dimensiones y formato
        """
        # Preparar datos
        temporal_data = self._prepare_date_data()

        # Crear diccionario de nombres cortos para los temas
        theme_shortcuts = {
            "Derechos humanos, Inclusión, discriminación, social, impacto": "Derechos Humanos e Inclusión",
            "Transparencia, datos input": "Transparencia y Datos",
            "Seguridad, privacidad, datos personales, regulación": "Seguridad y Privacidad",
            "HumanoMáquina": "Interacción Humano-Máquina",
            "Beneficios y usos": "Beneficios y Usos",
            "Risks": "Risks"
        }

        # Aplicar nombres cortos
        temporal_data['theme_short'] = temporal_data['theme'].map(theme_shortcuts)

        # Importar plotly
        import plotly.express as px

        # Crear bubble chart
        fig = px.scatter(
            temporal_data,
            x='year',
            y='theme_short',  # Usar nombres cortos
            size='count',
            color='theme_short',  # Usar nombres cortos para colores
            title='Distribución de artículos por año y tema',
            labels={
                'count': 'Cantidad de artículos',
                'year': 'Año',
                'theme_short': 'Tema'  # Actualizar etiqueta
            },
            size_max=60,  # Aumentar tamaño máximo de las burbujas
            hover_data={
                'theme_short': False,  # No mostrar en hover
                'count': True,
                'year': True
            },
            hover_name='theme_short'  # Mostrar tema en el título del hover
        )

        # Ajustar layout
        fig.update_layout(
            height=800,
            width=1200,  # Especificar ancho
            showlegend=True,
            title_x=0.5,
            title_y=0.95,
            margin=dict(l=20, r=20, t=80, b=20),  # Ajustar márgenes
            xaxis=dict(
                tickmode='linear',
                tick0=temporal_data['year'].min(),
                dtick=1,
                range=[
                    temporal_data['year'].min()-0.5,
                    temporal_data['year'].max()+0.5
                ]
            ),
            yaxis=dict(
                title_font=dict(size=14),  # Aumentar tamaño de fuente
                tickfont=dict(size=12)     # Aumentar tamaño de fuente
            ),
            legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="left",
                x=1.02,
                font=dict(size=12)
            ),
            hoverlabel=dict(
                bgcolor="white",
                font_size=14
            )
        )

        # Guardar como HTML
        fig.write_html(
            str(self.output_dir / 'temporal_bubble_chart.html'),
            include_plotlyjs='cdn',  # Usar CDN para plotly.js
            full_html=True
        )

    def create_theme_evolution_line(self):
        """
        Crea un gráfico de líneas que muestra la evolución temporal de cada tema
        """
        temporal_data = self._prepare_date_data()

        # Crear figura
        plt.figure(figsize=(15, 8))

        # Crear líneas para cada tema
        for theme in temporal_data['theme'].unique():
            theme_data = temporal_data[temporal_data['theme'] == theme]
            plt.plot(theme_data['year'],
                    theme_data['count'],
                    marker='o',
                    linewidth=2,
                    label=theme)

        # Ajustar ejes y etiquetas
        plt.title('Evolución Temporal del Número de Artículos por Tema', pad=20, fontsize=14)
        plt.xlabel('Año', fontsize=12)
        plt.ylabel('Número de Artículos', fontsize=12)

        # Rotar etiquetas del eje x
        plt.xticks(rotation=45)

        # Agregar leyenda
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

        # Agregar grid
        plt.grid(True, linestyle='--', alpha=0.7)

        # Ajustar layout
        plt.tight_layout()

        # Guardar figura
        plt.savefig(self.output_dir / 'theme_evolution.png',
                   dpi=300,
                   bbox_inches='tight')
        plt.close()

    def create_all_visualizations(self):
        """
        Crea todas las visualizaciones
        """
        print("Creando visualizaciones...")

        print("1. Generando gráfico de distribución por tema...")
        self.create_theme_distribution_bar()

        print("2. Generando gráfico de porcentajes...")
        self.create_theme_percentage_pie()

        print("3. Generando gráfico de distribución temporal...")
        self.create_temporal_bubble_chart()

        print("4. Generando gráfico de evolución temporal...")
        self.create_theme_evolution_line()

        print(f"\nVisualizaciones guardadas en el directorio: {self.output_dir}")
        print("Archivos generados:")
        print("- theme_distribution.png: Gráfico de barras con la distribución por tema")
        print("- theme_percentage.png: Gráfico circular con porcentajes")
        print("- temporal_distribution.png: Gráfico de distribución temporal")
        print("- theme_evolution.png: Gráfico de evolución temporal por tema")

def main():
    # Inicializar visualizador
    visualizer = NewsVisualizer(
        'news/reclassified_analysis_full_with_dates2.csv',
        'news/reclassified_analysis_theme_analysis.csv'
    )

    # Generar todas las visualizaciones
    visualizer.create_all_visualizations()

if __name__ == "__main__":
    main()

# Eda Foros

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pathlib import Path

class RedditVisualizer:
    def __init__(self, full_file, theme_analysis_file):
        self.full_df = pd.read_csv(full_file)
        self.theme_analysis = pd.read_csv(theme_analysis_file)
        self.output_dir = Path('visualizations')
        self.output_dir.mkdir(exist_ok=True)

        self.theme_shortcuts = {
            "Derechos humanos, Inclusión, discriminación, social, impacto": "Derechos Humanos e Inclusión",
            "Transparencia, datos input": "Transparencia y Datos",
            "Seguridad, privacidad, datos personales, regulación": "Seguridad y Privacidad",
            "HumanoMáquina": "Interacción Humano-Máquina",
            "Beneficios y usos": "Beneficios y Usos",
            "Risks": "Risks"
        }

    def create_distribution_bar(self):
        """Bar chart de distribución por tema"""
        # Asegurarse de que los datos están correctamente formateados
        data = (self.full_df.groupby('theme')
               .size()
               .reset_index(name='count')
               .sort_values('count', ascending=True))

        # Aplicar nombres cortos
        data['theme_short'] = data['theme'].map(self.theme_shortcuts)

        plt.figure(figsize=(12, 8))
        bars = plt.barh(data['theme_short'],
                       data['count'],
                       color=sns.color_palette("husl", len(data)))

        # Añadir etiquetas
        for i, bar in enumerate(bars):
            width = bar.get_width()
            plt.text(width, i, f' {int(width)}',
                    va='center', ha='left')

        plt.title('Distribución de Artículos por Tema', pad=20, fontsize=14)
        plt.xlabel('Número de Artículos', fontsize=12)
        plt.ylabel('Tema', fontsize=12)

        plt.tight_layout()
        plt.savefig(self.output_dir / 'theme_distribution.png',
                   dpi=300, bbox_inches='tight')
        plt.close()

    def create_percentage_pie(self):
        """Pie chart de porcentajes"""
        # Calcular porcentajes
        data = (self.full_df.groupby('theme')
               .size()
               .reset_index(name='count'))
        total = data['count'].sum()
        data['percentage'] = (data['count'] / total) * 100

        # Aplicar nombres cortos
        data['theme_short'] = data['theme'].map(self.theme_shortcuts)

        # Ordenar por porcentaje
        data = data.sort_values('percentage', ascending=False)

        plt.figure(figsize=(12, 8))
        wedges, texts, autotexts = plt.pie(data['percentage'],
                                         labels=data['theme_short'],
                                         autopct='%1.1f%%',
                                         colors=sns.color_palette("husl", len(data)))

        # Ajustar propiedades del texto
        plt.setp(autotexts, size=8, weight="bold")
        plt.setp(texts, size=8)

        plt.title('Distribución Porcentual de Artículos por Tema',
                 pad=20, fontsize=14)

        plt.tight_layout()
        plt.savefig(self.output_dir / 'theme_percentage.png',
                   dpi=300, bbox_inches='tight')
        plt.close()

    def create_bubble_chart(self):
        """Bubble chart interactivo con Plotly"""
        # Preparar datos temporales
        self.full_df['year'] = pd.to_datetime(self.full_df['publish_date']).dt.year
        temporal_data = (self.full_df.groupby(['year', 'theme'])
                        .size()
                        .reset_index(name='count'))

        # Aplicar nombres cortos
        temporal_data['theme_short'] = temporal_data['theme'].map(self.theme_shortcuts)

        fig = px.scatter(temporal_data,
                        x='year',
                        y='theme_short',
                        size='count',
                        color='theme_short',
                        title='Distribución de artículos por año y tema',
                        labels={'count': 'Cantidad de artículos',
                               'year': 'Año',
                               'theme_short': 'Tema'},
                        size_max=50)

        fig.update_layout(
            height=800,
            showlegend=True,
            title_x=0.5,
            xaxis=dict(
                tickmode='linear',
                tick0=temporal_data['year'].min(),
                dtick=1,
                range=[temporal_data['year'].min()-0.5,
                       temporal_data['year'].max()+0.5]
            )
        )

        fig.write_html(str(self.output_dir / 'temporal_bubble_chart.html'))

    def create_all_visualizations(self):
        print("Creando visualizaciones...")
        self.create_distribution_bar()
        print("✓ Gráfico de distribución creado")
        self.create_percentage_pie()
        print("✓ Gráfico de porcentajes creado")
        self.create_bubble_chart()
        print("✓ Bubble chart creado")
        print(f"\nVisualizaciones guardadas en: {self.output_dir}")

def main():
    visualizer = RedditVisualizer(
        'forums/forums_full_with_dates.csv',
        'forums/forums_reclassified_theme_analysis.csv'
    )
    visualizer.create_all_visualizations()

if __name__ == "__main__":
    main()

# Cargar el dataframe antes de generar las visualizaciones sankey

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from collections import defaultdict

def create_sankey_diagram(df, min_connections=5):  # Añadimos umbral mínimo
    theme_shortcuts = {
        "Derechos humanos, Inclusión, discriminación, social, impacto": "Derechos Humanos e Inclusión",
            "Transparencia, datos input": "Transparencia y Datos",
            "Seguridad, privacidad, datos personales, regulación": "Seguridad y Privacidad",
            "HumanoMáquina": "Interacción Humano-Máquina",
            "Beneficios y usos": "Beneficios y Usos",
            "Risks": "Risks"
    }

    # Recolectar conexiones
    article_themes = defaultdict(list)
    for _, row in df.iterrows():
        themes = row['theme'].split(' . ')
        short_themes = [theme_shortcuts.get(theme, theme) for theme in themes]
        article_themes[row['url']].extend(short_themes)

    # Contar conexiones y temas
    theme_connections = defaultdict(int)
    theme_counts = defaultdict(int)

    for themes in article_themes.values():
        # Contar frecuencia individual
        for theme in themes:
            theme_counts[theme] += 1

        # Crear conexiones
        if len(themes) > 1:
            for i in range(len(themes)):
                for j in range(i + 1, len(themes)):
                    theme1, theme2 = sorted([themes[i], themes[j]])
                    theme_connections[(theme1, theme2)] += 1

    # Filtrar conexiones por umbral mínimo
    significant_connections = {k: v for k, v in theme_connections.items() if v >= min_connections}

    # Obtener temas que tienen conexiones significativas
    connected_themes = set()
    for (source, target) in significant_connections.keys():
        connected_themes.add(source)
        connected_themes.add(target)

    # Crear índices solo para temas conectados
    all_themes = sorted(list(connected_themes))
    theme_to_index = {theme: idx for idx, theme in enumerate(all_themes)}

    # Preparar datos para Sankey
    sources = []
    targets = []
    values = []

    for (source, target), value in significant_connections.items():
        sources.append(theme_to_index[source])
        targets.append(theme_to_index[target])
        values.append(value)

    # Colores más suaves
    node_colors = [
        '#FF9999', '#66B2FF', '#99FF99', '#FFCC99',
        '#FF99CC', '#99CCFF', '#FFB366', '#99FF99'
    ]

    # Crear figura con diseño mejorado
    fig = go.Figure(data=[go.Sankey(
        arrangement = "snap",  # Cambiado a "snap" para mejor disposición
        node = dict(
            pad = 20,
            thickness = 20,
            line = dict(color = "black", width = 0.5),
            label = all_themes,
            color = node_colors[:len(all_themes)],
            customdata = [theme_counts[theme] for theme in all_themes],
            hovertemplate = 'Tema: %{label}<br>Total de artículos: %{customdata}<extra></extra>'
        ),
        link = dict(
            source = sources,
            target = targets,
            value = values,
            hovertemplate = 'De %{source.label}<br>' +
                          'A %{target.label}<br>' +
                          'Artículos compartidos: %{value}<extra></extra>',
            color = ['rgba(150,150,150,0.3)'] * len(values)  # Conexiones más sutiles
        )
    )])

    fig.update_layout(
        title = dict(
            text = "Conexiones entre Temas<br>" +
                   f"<sup>Mostrando conexiones con {min_connections} o más artículos compartidos</sup>",
            font = dict(size=20)
        ),
        font_size = 12,
        height = 800,
        width = 1200,
        plot_bgcolor = 'white',
        paper_bgcolor = 'white'
    )

    return fig

# Crear y mostrar el diagrama
fig = create_sankey_diagram(df, min_connections=10)  # Ajusta este valor según necesites
fig.show()

# Guardar como HTML
fig.write_html("tema_conexiones_sankey.html")